This project is meant to provide practice using SQLite databases. We will use data on countries across the world from CIA factbook from 2015.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [4]:
# Query which tables are in the database
%%sql
Select * from sqlite_master where type='table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [13]:
%%sql
Select * from facts limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Let's find the minimum and maximum population and population growth.

In [22]:
%%sql
select min(population) as "Min Population", 
max(population) "Max Population", 
min(population_growth) "Min Population Growth",
max(population_growth) "Max Population Growth"
from facts

Done.


Min Population,Max Population,Min Population Growth,Max Population Growth
0,7256490011,0.0,4.02


The population minimum and maximum seem strange. Let's figure out which countries they are coming from.

In [38]:
%%sql
select name from facts where population = (select min(population) from facts) 

Done.


name
Antarctica


In [40]:
%%sql
select name from facts where population = (select max(population) from facts)

Done.


name
World


Confirming that these are in fact reasonable values since they come from Antarctica and the World, we can continue by checking the average population and size of the countries.

In [43]:
%%sql
select round(avg(population), 2) "avg pop", round(avg(area_land), 2) "avg size" from facts

Done.


avg pop,avg size
62094928.32,522702.58


In [44]:
%%sql
select name "Above Average Population" from facts where population > (select avg(population) from facts)

Done.


name
Bangladesh
Brazil
China
"Congo, Democratic Republic of the"
Egypt
Ethiopia
France
Germany
India
Indonesia


In [48]:
%%sql
select name "Above Average Size" from facts where area_land > (select avg(area_land) from facts)

Done.


Above Average Size
Afghanistan
Algeria
Angola
Argentina
Australia
Bolivia
Botswana
Brazil
Burma
Canada


After seeing which countries have higher than average population and land mass, let's look deeper at populations. First let's see which countries have the highest and lowest birth rates.

In [55]:
%%sql
select name, birth_rate from facts order by birth_rate desc limit 10

Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58


In [60]:
%%sql
select name, birth_rate from facts where birth_rate != "None" 
order by birth_rate limit 10

Done.


name,birth_rate
Monaco,6.65
Saint Pierre and Miquelon,7.42
Japan,7.93
Andorra,8.13
"Korea, South",8.19
Singapore,8.27
Slovenia,8.42
Germany,8.47
Taiwan,8.47
San Marino,8.63


Let's compare with death rates and see which countries are in negative growth.

In [65]:
%%sql
select name, death_rate from facts 
where death_rate != "None" order by death_rate desc limit 10 

Done.


name,death_rate
Lesotho,14.89
Ukraine,14.46
Bulgaria,14.44
Guinea-Bissau,14.33
Latvia,14.31
Chad,14.28
Lithuania,14.27
Namibia,13.91
Afghanistan,13.89
Central African Republic,13.8


In [73]:
%%sql
select name, birth_rate, death_rate, birth_rate - death_rate difference 
from facts where death_rate > birth_rate 
order by difference

Done.


name,birth_rate,death_rate,difference
Bulgaria,8.92,14.44,-5.52
Serbia,9.08,13.66,-4.58
Latvia,10.0,14.31,-4.3100000000000005
Lithuania,10.1,14.27,-4.17
Ukraine,10.72,14.46,-3.74
Hungary,9.16,12.73,-3.5700000000000003
Germany,8.47,11.42,-2.9499999999999993
Slovenia,8.42,11.37,-2.9499999999999993
Romania,9.14,11.9,-2.76
Croatia,9.45,12.18,-2.7300000000000004
